In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
import torch
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
plt.style.use('dark_background')
import data
from skorch.callbacks import Checkpoint
from skorch import NeuralNetRegressor
#import models
from tqdm import tqdm
import train_reg
import config
import pandas as pd
import features
from scipy.stats import skew, pearsonr
import outcomes
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.nn import functional as F

/accounts/grad/xsli/.local/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/accounts/grad/xsli/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
dset = 'clath_aux+gak_a7d2_new'
df = data.get_data(dset=dset)

## add regression response, downsample tracks

In [4]:
length = 40
df = train_reg.add_sig_mean(df) # regression response
df['X_same_length'] = [features.downsample(df.iloc[i]['X'], length) for i in range(len(df))] # downsampling

## extract feature names, get training data

In [5]:
feat_names = data.get_feature_names(df)
feat_names = [x for x in feat_names 
                      if not x.startswith('sc_') 
                      and not x.startswith('nmf_')
                      and not x in ['center_max', 'left_max', 'right_max', 'up_max', 'down_max',
                                   'X_max_around_Y_peak', 'X_max_after_Y_peak', 'X_quantiles',
                                   'X_d1', 'X_d2', 'X_d3', 'slope_end', 'X_same_length'
                                   ]
                      and not x.startswith('pc_')
                      and not 'log' in x
                      and not 'binary' in x
                     ]

In [6]:
df_train = df[df.cell_num.isin(config.DSETS[dset]['train'])] 
df_train = df_train[df_train.lifetime > 15]
df_train_nonna = df_train[feat_names + ['X_same_length', 'cell_num', 'Y_sig_mean']]
df_train_nonna = df_train_nonna.dropna() 

## train fully connected network

In [ ]:
out_dir = f'results/regression/deep_learning/Dec4'
os.makedirs(out_dir, exist_ok=True)
outcome_def = 'Y_sig_mean'
train_reg.train_reg(df_train_nonna, 
                    feat_names=['X_same_length'] + feat_names,#['lifetime', 'X_max', 'mean_square_displacement'], 
                    model_type='fcnn', 
                    outcome_def=outcome_def,
                    out_name=oj(out_dir, f'{dset}_{outcome_def}_fcnn.pkl'),
                    fcnn_hidden_neurons=20,
                    fcnn_epochs=2000)

0it [00:00, ?it/s]

Looping over cv...
Epoch: 99, Average loss: 1259.0460122699387
Epoch: 199, Average loss: 1185.1182229990013
Epoch: 299, Average loss: 1095.6385896704237
Epoch: 399, Average loss: 1126.8110465116279
Epoch: 499, Average loss: 1134.464055143387
Epoch: 599, Average loss: 1278.131054715366
Epoch: 699, Average loss: 1217.4097321301185
Epoch: 799, Average loss: 1041.2802111570838
Epoch: 899, Average loss: 1171.6033314310173
Epoch: 999, Average loss: 1053.4302593094592
Epoch: 1099, Average loss: 1402.9605863889285
Epoch: 1199, Average loss: 1022.9412897702954
Epoch: 1299, Average loss: 1102.0082304893708
Epoch: 1399, Average loss: 1061.5520937366243
Epoch: 1499, Average loss: 1187.0803787986874
Epoch: 1599, Average loss: 1097.4739442145813
Epoch: 1699, Average loss: 1095.2220448708804
Epoch: 1799, Average loss: 1098.5336531602227
Epoch: 1899, Average loss: 1240.5013910686262


1it [06:52, 412.39s/it]

Epoch: 1999, Average loss: 1205.6215134113284
Epoch: 99, Average loss: 1267.6155188106795
Epoch: 199, Average loss: 1211.9478724211165
Epoch: 299, Average loss: 1181.5198346480583
Epoch: 399, Average loss: 1152.7353326001214
Epoch: 499, Average loss: 1153.2442828428398
Epoch: 599, Average loss: 1129.4239608616506
Epoch: 699, Average loss: 1126.9722485588593
Epoch: 799, Average loss: 1119.879257054005
Epoch: 899, Average loss: 1102.3409246055826
Epoch: 999, Average loss: 1094.0667949787621
Epoch: 1099, Average loss: 1102.6873008950242
Epoch: 1199, Average loss: 1094.686608768204
Epoch: 1299, Average loss: 1086.8673676425972
Epoch: 1399, Average loss: 1083.0529808859224
Epoch: 1499, Average loss: 1084.8435983009708
Epoch: 1599, Average loss: 1068.981179839199
Epoch: 1699, Average loss: 1066.926843143204
Epoch: 1799, Average loss: 1066.3060243476941
Epoch: 1899, Average loss: 1060.7195084951456


2it [13:16, 403.85s/it]

...

Epoch: 1999, Average loss: 948.6257537873216
Epoch: 99, Average loss: 1277.5727480834753
Epoch: 199, Average loss: 1216.1740310902896
Epoch: 299, Average loss: 1158.460311967632
Epoch: 399, Average loss: 1195.922558205565
Epoch: 499, Average loss: 1130.9667891113004
Epoch: 599, Average loss: 1134.2415974588303
Epoch: 699, Average loss: 1110.2924297274276
Epoch: 799, Average loss: 1054.9545446479274
Epoch: 899, Average loss: 1103.170038330494
Epoch: 999, Average loss: 1192.5332197614991
Epoch: 1099, Average loss: 1053.8106899488928
Epoch: 1199, Average loss: 1082.7428219051676
Epoch: 1299, Average loss: 1069.918609809767
Epoch: 1399, Average loss: 1081.7548889125496
Epoch: 1499, Average loss: 1057.5871841283363
Epoch: 1599, Average loss: 1059.6614494605337
Epoch: 1699, Average loss: 1099.167305508234
Epoch: 1799, Average loss: 1136.995838657013
Epoch: 1899, Average loss: 1033.194962024418


6it [40:08, 403.49s/it]

Epoch: 1999, Average loss: 1192.8589224872233
Epoch: 99, Average loss: 1263.8684864827007
Epoch: 199, Average loss: 1229.7576691413362
Epoch: 299, Average loss: 1201.404197016389
Epoch: 399, Average loss: 1130.5830648550218
Epoch: 499, Average loss: 1133.6304454405379
Epoch: 599, Average loss: 1110.3781779661017
Epoch: 699, Average loss: 1111.517229303824
Epoch: 799, Average loss: 1134.5860239529345
Epoch: 899, Average loss: 1112.335778820563
Epoch: 999, Average loss: 1101.6599401176636
Epoch: 1099, Average loss: 1131.1528575430732
Epoch: 1199, Average loss: 1118.1939784983892
Epoch: 1299, Average loss: 1133.1182501050566
Epoch: 1399, Average loss: 1101.0188051547836
Epoch: 1499, Average loss: 1126.0994361955456
Epoch: 1599, Average loss: 1090.3355336881916
Epoch: 1699, Average loss: 1096.3968255357893
Epoch: 1799, Average loss: 1063.6780011206051
Epoch: 1899, Average loss: 1057.5742313349208


7it [47:08, 408.34s/it]

Epoch: 1999, Average loss: 1056.257599103516
Epoch: 99, Average loss: 1255.0078614008942
Epoch: 199, Average loss: 1199.517976900149
Epoch: 299, Average loss: 1157.1555141579731
Epoch: 399, Average loss: 1128.9180234724292
Epoch: 499, Average loss: 1123.42185171386
Epoch: 599, Average loss: 1117.2181725037258
Epoch: 699, Average loss: 1108.9489474664679
Epoch: 799, Average loss: 1099.1657786885246
Epoch: 899, Average loss: 1086.558830104322
